In [1]:
from __future__ import division
import pandas as pd
import numpy as np
import json
import os,sys
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import numpy as np

In [6]:

pd.options.display.float_format = '{:,.2f}'.format
dataset_base_dir = './'
dataset_file_name = 'lsac.csv'

In [36]:

file_path = os.path.join(dataset_base_dir,dataset_file_name)
with open(file_path, "r") as file_name:
  temp_df = pd.read_csv(file_name)

# Columns of interest  
df = temp_df[['锘縇SAT','GPA','resident','Year','Gender','Race','URM','admit']].copy()
renameColumns={'Gender':'sex','锘縇SAT': 'LSAT'}
target_variable = 'admit'
target_value = 1

# Renaming columns
df = df.rename(columns = renameColumns)
columns = renameColumns.values()

# NaN in 'pass_bar' refer to dropouts. Considering NaN as failing the bar.
df['admit'] = df['admit'].fillna(value=0.0)
df['admit'] = df.apply(lambda x: 1 if x['admit']==1.0 else '0', axis=1).astype('category')

df['GPA'] = df['GPA'].fillna(value=0.0)
df['Race'] = df['Race'].replace(' ','other')
df = df.dropna()


# Process protected-column values
df['sex'] = df['sex'].astype('category')

In [48]:
train_df

LSAT  GPA  resident  Year  sex   Race  URM admit
37720 158.00 3.80      1.00  2006 0.00  White    0     0
29959 153.00 3.18      1.00  2005 1.00  White    0     1
15258 160.00 3.08      1.00  2007 1.00  other    0     0
63863 151.00 3.56      0.00  2005 1.00  other    0     0
29172 169.00 2.49      0.00  2007 1.00  White    0     1
...      ...  ...       ...   ...  ...    ...  ...   ...
62741 155.00 3.06      0.00  2005 1.00  White    0     0
38161 169.00 3.71      0.00  2006 1.00  White    0     0
860   164.00 3.80      0.00  2006 1.00  White    0     1
15796 169.00 3.63      0.00  2007 1.00  other    0     1
56536 147.00 3.18      1.00  2008 0.00  White    0     0

[45715 rows x 8 columns]

In [38]:

train_df, test_df = train_test_split(df, test_size=0.30, random_state=42)

output_file_path = os.path.join(dataset_base_dir,'train.csv')
with open(output_file_path, mode="w") as output_file:
    train_df.to_csv(output_file,index=False,columns=columns,header=False)
    output_file.close()

output_file_path = os.path.join(dataset_base_dir,'test.csv')
with open(output_file_path, mode="w") as output_file:
    test_df.to_csv(output_file,index=False,columns=columns,header=False)
    output_file.close()

In [39]:

IPS_example_weights_without_label = {
  0: (len(train_df))/(len(train_df[(train_df.Race != 'Black') & (train_df.sex != 0)])), # 00: White Male
  1: (len(train_df))/(len(train_df[(train_df.Race != 'Black') & (train_df.sex == 0)])), # 01: White Female
  2: (len(train_df))/(len(train_df[(train_df.Race == 'Black') & (train_df.sex != 0)])), # 10: Black Male
  3: (len(train_df))/(len(train_df[(train_df.Race == 'Black') & (train_df.sex == 0)]))  # 11: Black Female
}
  
output_file_path = os.path.join(dataset_base_dir,'IPS_example_weights_without_label.json')
with open(output_file_path, mode="w") as output_file:
    output_file.write(json.dumps(IPS_example_weights_without_label))
    output_file.close()

print(IPS_example_weights_without_label)

{0: 1.8700400883580135, 1: 2.613629866788634, 2: 30.31498673740053, 3: 20.13876651982379}


In [45]:

IPS_example_weights_with_label = {
0: (len(train_df))/(len(train_df[(train_df[target_variable] != target_value) & (train_df.Race != 'Black') & (train_df.sex != 0)])), # 000: Negative White Male
1: (len(train_df))/(len(train_df[(train_df[target_variable] != target_value) & (train_df.Race != 'Black') & (train_df.sex == 0)])), # 001: Negative White Female
2: (len(train_df))/(len(train_df[(train_df[target_variable] != target_value) & (train_df.Race == 'Black') & (train_df.sex != 0)])), # 010: Negative Black Male
3: (len(train_df))/(len(train_df[(train_df[target_variable] != target_value) & (train_df.Race == 'Black') & (train_df.sex == 0)])), # 011: Negative Black Female
4: (len(train_df))/(len(train_df[(train_df[target_variable] == target_value) & (train_df.Race != 'Black') & (train_df.sex != 0)])), # 100: Positive White Male
5: (len(train_df))/(len(train_df[(train_df[target_variable] == target_value) & (train_df.Race != 'Black') & (train_df.sex == 0)])), # 101: Positive White Female
6: (len(train_df))/(len(train_df[(train_df[target_variable] == target_value) & (train_df.Race == 'Black') & (train_df.sex != 0)])), # 110: Positive Black Male
7: (len(train_df))/(len(train_df[(train_df[target_variable] == target_value) & (train_df.Race == 'Black') & (train_df.sex == 0)])), # 111: Positive Black Female
}
  
output_file_path = os.path.join(dataset_base_dir,'IPS_example_weights_with_label.json')
with open(output_file_path, mode="w") as output_file:
    output_file.write(json.dumps(IPS_example_weights_with_label))
    output_file.close()

print(IPS_example_weights_with_label)

{0: 2.471749121384158, 1: 3.5347560504136704, 2: 38.25523012552301, 3: 25.090559824368825, 4: 7.681902201310704, 5: 10.02961825362001, 6: 146.05431309904154, 7: 102.04241071428571}


In [46]:
cat_cols = train_df.select_dtypes(include='category').columns
vocab_dict = {}
for col in cat_cols:
  vocab_dict[col] = list(set(train_df[col].cat.categories))
  
output_file_path = os.path.join(dataset_base_dir,'vocabulary.json')
with open(output_file_path, mode="w") as output_file:
    output_file.write(json.dumps(vocab_dict))
    output_file.close()
print(vocab_dict)

{'sex': [0.0, 1.0], 'admit': [1, '0']}


In [47]:
temp_dict = train_df.describe().to_dict()
mean_std_dict = {}
for key, value in temp_dict.items():
  mean_std_dict[key] = [value['mean'],value['std']]

output_file_path = os.path.join(dataset_base_dir,'mean_std.json')
with open(output_file_path, mode="w") as output_file:
    output_file.write(json.dumps(mean_std_dict))
    output_file.close()
print(mean_std_dict)

{'LSAT': [155.80660614677896, 8.368060648136261], 'GPA': [3.322078496792234, 0.4437985665906993], 'resident': [0.3150607021765285, 0.46454512882971627], 'URM': [0.1496007874876955, 0.3566835780271116]}
